In [1]:
import numpy as np

def cosine_similarity(vec_a: np.ndarray, vec_b: np.ndarray)->float:
  dot = np.dot(vec_a, vec_b)
  norm_a = np.linalg.norm(vec_a)
  norm_b = np.linalg.norm(vec_b)
  return dot / (norm_a*norm_b)

# openai embedding

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from openai import OpenAI
openai_client = OpenAI()

In [4]:
def get_openai_embedding(text:str, model='text-embedding-3-small'):
  response = openai_client.embeddings.create(
    input = text, 
    model=model
  )
  return response.data[0].embedding

In [6]:
text_str = '안녕하세요'
emb_vector = get_openai_embedding(text_str)
len(emb_vector)

1536

In [7]:
# 임베딩모델 생성
from langchain_upstage import UpstageEmbeddings
import os
embeddings = UpstageEmbeddings(
  api_key=os.getenv('UPSTAGE_API_KEY'),
  model='solar-embedding-1-large'
)

In [8]:
#업스테이지 모델로 임베딩함수
def get_upstage_embedding(text: str, is_query: bool = False) -> np.ndarray:
    if is_query:
        vec = embeddings.embed_query(text)
    else:
        vec = embeddings.embed_documents([text])[0]  # 리스트에서 첫 번째 요소만 추출
    return np.array(vec)

In [9]:
# pip install -qU langchain-core langchain-upstage
import os

from langchain_upstage import UpstageEmbeddings
 
embeddings = UpstageEmbeddings(
    api_key= os.getenv('UPSTAGE_API_KEY'),
    model="embedding-query"
)
 
doc_result = embeddings.embed_documents(
    ["Sam is a teacher.", "This is another document"]
)
print(doc_result)
 
query_result = embeddings.embed_query("What does Sam do?")
print(query_result)

[[0.016357421875, 0.0171356201171875, -0.0077972412109375, 0.024078369140625, 0.00342559814453125, -0.00682830810546875, -0.01465606689453125, -0.01067352294921875, -0.015625, 0.00673675537109375, 0.0183258056640625, 0.00507354736328125, 0.007244110107421875, 0.01177978515625, 0.02777099609375, 0.02191162109375, -0.0213623046875, -0.0012531280517578125, -0.0030117034912109375, -0.0167388916015625, -0.02410888671875, -0.0090789794921875, -0.01122283935546875, -0.005924224853515625, -0.01036834716796875, 0.0162811279296875, 0.002777099609375, -0.01197052001953125, 0.00518798828125, 0.02154541015625, 0.0056610107421875, 0.01412200927734375, 0.0020542144775390625, -0.0094146728515625, 0.01070404052734375, -0.01226806640625, -0.003635406494140625, 0.02374267578125, -0.012420654296875, 0.01287078857421875, -0.0112152099609375, -0.02001953125, -0.01410675048828125, 8.320808410644531e-05, -0.016510009765625, 0.012786865234375, 0.00691986083984375, -0.0008373260498046875, 0.004840850830078125, 

In [10]:
len(query_result)

4096

#openai embedding 모델로 임베딩

In [11]:
texts = ['king', 'queen', 'slave', '왕']

In [12]:
openai_embeddings = {txt: get_openai_embedding(txt) for txt in texts}

In [13]:
cosine_similarity(openai_embeddings['queen'], openai_embeddings['king'])

np.float64(0.590601530239691)

#업스테이 임베딩 모델로 임베딩

In [14]:
upstage_embeddings = {txt: get_upstage_embedding(txt) for txt in texts}

In [15]:
cosine_similarity(upstage_embeddings['queen'], upstage_embeddings['king'])

np.float64(0.6446770680612558)

# 왕의 비교

In [16]:
cosine_similarity(openai_embeddings['왕'], openai_embeddings['king'])

np.float64(0.5039512482944454)

In [17]:
cosine_similarity(upstage_embeddings['왕'], upstage_embeddings['king'])

np.float64(0.6960682511235363)

젬마2 모델로 임베딩 비교

# 올라마 임베딩 > huggingface 의 임베딩 모델

In [18]:
!pip install sentence-transformers

  Using cached sentence_transformers-5.1.0-py3-none-any.whl.metadata (16 kB)
  Using cached tokenizers-0.22.0-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.6.2-cp38-abi3-win_amd64.whl.metadata (4.1 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached sentence_transformers-5.1.0-py3-none-any.whl (483 kB)
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ------------------------- -------------- 7.3/11.6 MB 37.8 MB/s eta 0:00:01
   ---------------------------------------- 11.6/11.6 MB 33.1 MB/s eta 0:00:00
Using cached tokenizers-0.22.0-cp39-abi3-win_amd64.whl (2.7 MB)
Using cached safetensors-0.6.2-cp38-abi3-win_amd64.whl (320 kB)
   ---------------------------------------- 0.0/241.3 MB ? eta -:--:--
    --------------------------------------- 5.5/241.3 MB 25.8 MB/s eta 0:00:10
   - --

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-upstage 0.7.3 requires tokenizers<0.21.0,>=0.20.0, but you have tokenizers 0.22.0 which is incompatible.


In [19]:
from langchain_community.chat_models import ChatOllama
llm_ollama = ChatOllama(model='gemma2')
response = llm_ollama.invoke('안녕? 네 소개를 2줄로 작성해')
response.content

C:\Users\Admin\AppData\Local\Temp\ipykernel_3432\3721171275.py:2: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm_ollama = ChatOllama(model='gemma2')


'안녕하세요! 저는 구글 AI에서 훈련된 대화형 AI입니다.\n\n다양한 질문에 답변하고, 이야기 나누고, 창의적인 작업을 도와드릴 수 있습니다.'

# OS 계열의 임베딩 모델 사용
허깅페이스의 임베딩 모델 사용 - transformers 라이브러리, GPU 기반 pytorch
토치쿠다 기반의 가상환경에서 실행 

In [1]:
from sentence_transformers import SentenceTransformer
import torch
#MODEL = 'BAAI/bge-multilingual-gemma2'
MODEL = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

c:\Users\Admin\miniconda3\envs\prompting_env\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]


'cpu'

In [2]:
model = SentenceTransformer(MODEL, device=device)

c:\Users\Admin\miniconda3\envs\prompting_env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--sentence-transformers--paraphrase-multilingual-MiniLM-L12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package i

In [3]:
texts = ['king', 'queen', 'slave', '왕']

In [4]:
model.max_seq_length=256
gemma_embedding = model.encode(texts, batch_size=64, normalize_embeddings=True, convert_to_numpy=True, show_progress_bar=True)
gemma_embedding

Batches: 100%|██████████| 1/1 [00:00<00:00,  1.03it/s]


array([[-0.00158092,  0.10178691, -0.01695298, ..., -0.05791414,
        -0.04557932, -0.04427629],
       [ 0.06440255, -0.00555204,  0.02317658, ..., -0.05347716,
         0.00656337, -0.04465976],
       [-0.049297  ,  0.07381599, -0.00779097, ..., -0.08466545,
        -0.04895947, -0.06428003],
       [ 0.00818062,  0.07960667, -0.02314878, ..., -0.04931473,
        -0.04339726, -0.00256805]], shape=(4, 384), dtype=float32)